# Model Notebook

#### Import libraries section

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from dotenv import load_dotenv
import os

import sys
sys.path.append("/Users/zemariatrindade/BTS/Financy_App/Scripts_and_Data/venv/lib/python3.11/site-packages")
import psycopg2

#### Importing env variables

In [ ]:
# Load environment variables from .env file
load_dotenv()

dbname = os.getenv('db_name')
user = os.getenv('db_username')
password = os.getenv('db_password')
host = os.getenv('db_host')
port = os.getenv('db_port')

#### Importing anomalies found with reseach to annotate our dataset

In [ ]:
anomalies_df = pd.read_csv("Anomalies_tracker.csv")
anomalies_df['cik_str'] = anomalies_df['cik_str'].astype(str).str.zfill(10)

In [ ]:
anomalies_df

##### Making sure we don't have the same anomaly repeated
In the Anomalies_tracker, we had different tickers for the same cik number

In [ ]:
anomalies_df = anomalies_df.drop_duplicates(subset=["cik_str","Scandal Year"])

##### filtering for the years and forms format where we have data to match

In [ ]:
anomaly_cases = anomalies_df[(anomalies_df["XBRL format"]=="Yes") & (anomalies_df["10-Q/ 10-K available"]=="Yes")][["cik_str","Scandal Year","Anomaly Ticker"]]

In [ ]:
print("Number of cases:", len(anomaly_cases))
print("Number of tickers:", len(anomaly_cases["Anomaly Ticker"].unique()))

anomaly_cases

We have a total of 53 anomaly cases. We assume the company was anomalous the entire year. Given that we have more than 1 type of report per year, we will end up with more total anomalous entries.

These 53 anomaly cases represent 40 companies

#### Querying the aws RDS database
for the tickers we have found at least 1 anomaly record

In [ ]:
tuple_tickers = tuple(anomaly_cases["Anomaly Ticker"].unique())

In [ ]:
# Import the data
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)
cursor = conn.cursor()

query = f"""
    SELECT * 
    FROM edgar_selected_tickers_table
    WHERE ticker in {tuple_tickers};
"""
cursor.execute(query)
results = cursor.fetchall()

cursor.close()
conn.close()

In [ ]:
column_names = ['end_date', 'ticker', 'reporting_frame', 'form', 'account', 'unit', 'value']
query_df = pd.DataFrame(results, columns=column_names)

In [ ]:
query_df.ticker.nunique()

In [ ]:
query_df["year"] = query_df["end_date"].apply(lambda x: int(x.strftime("%Y")))
query_df = query_df.iloc[:,[0,7,1,2,3,4,5,6]]

In [ ]:
query_df.info()

In [ ]:
query_df.head()

In [ ]:
query_df.account.nunique()

In [ ]:
query_df.form.unique()

## Hypothesis Brach: Supervised model
Let's use the anomaly years (anomaly_cases) to match with the reports of those same years present in EDGAR.
And we will use the matches as binary classification cols.

1. we run a pivot table on top of the query to get the accounts as columns
2. we create a label column called "anomaly"

1. getting the accounts as columns

In [ ]:
pivoted_df = query_df.pivot_table(index=['end_date','year', 'ticker', 'reporting_frame', 'form', 'unit'], 
                            columns='account',
                            values='value',
                            aggfunc='sum',
                            fill_value=0)
# Fill any NaN values that might be created in the process.\We have to justify why 0 is a valid value.
pivoted_df = pivoted_df.reset_index()

In [ ]:
pivoted_df.head()

2. creating the label column "anomaly"

Assumptions:
- we will assign the value "1" to every date in the respective anomaly year
- we will use all type forms: 10-K, 10-Q, 8-K
- what types of anomalies are we considering? should we include all of them?

In [ ]:
merged_df = pivoted_df.merge(anomaly_cases, how="left",left_on=["ticker","year"], right_on=["Anomaly Ticker","Scandal Year"], indicator=True)
merged_df.drop(labels=["Scandal Year","Anomaly Ticker","cik_str"], axis=1, inplace=True)
merged_df['anomaly'] = (merged_df['_merge'] == 'both').astype(int)
merged_df.drop(labels=["_merge"], axis=1, inplace=True)

In [ ]:
merged_df.anomaly.value_counts()

In [ ]:
merged_df

#### Discovery: 8.7% of the values are anomalies
assuming we are using all the type forms and all forms are anomalous in a given anomalous year

### to-dos:
1. use karens data with the anomalies updated: same tickers, but different anomaly years >> only for netflix and deleted 
2. keep working on feature engineering
2. try models

## Feature Engineering
1. Handling Missing values - let's treat the null values as meaningful zeros.
2. Scaling and Normalization
3. Dimensionality Reduction
   1. PCA
   2. Feature selection with autoencoding
4. Encoding Categorical Variables
5. Generating Interaction Features:
6. Handling Imbalanced Data
7. Time-Based Features:
If the data is time-series in nature, consider generating lag features, rolling averages, or other time-based features.

### 1. Scaling
For the task of finding anomalies in a diverse dataset with varying scales and potential outliers, Robust Scaling is highly recommended due to its robustness to outliers. Since financial data often includes extreme values and outliers, Robust Scaling will ensure that these outliers do not disproportionately affect the scaling process, leading to better model performance and more reliable anomaly detection.

However, it can be beneficial to **experiment with both Standardization and Robust Scaling**, then evaluate model performance using cross-validation to determine the best approach for our specific dataset. 

In [ ]:
numerical_cols = list(merged_df.select_dtypes(include=['int', 'float']).columns)#
numerical_cols.remove("year")
numerical_cols.remove("anomaly")

In [ ]:
merged_df.unit.unique()

In [ ]:
merged_df.reporting_frame.unique()

In [ ]:
merged_df[merged_df.reporting_frame=="us-gaap"].unit.unique()

In [ ]:
merged_df[merged_df.reporting_frame=="dei"].unit.unique()

In [ ]:
query_df[query_df.unit=="pure"].account.unique()

### Modeling
1. Logistic regression
2. Tree-Based Models: Random forest and Gradient Boosting Machines (GBM): (e.g., XGBoost, LightGBM, CatBoost)
3. Support Vector Machines (SVM)
4. Neural Networks: MLPs and CNN, LSTMs
5. Ensemble Methods
6. Anomaly Detection Algorithms: Given the nature of anomalies, consider specialized anomaly detection techniques like Isolation Forest, One-Class SVM, or Autoencoders for anomaly detection.

In [ ]:
X=pivoted_df.iloc[:, 5:]

In [ ]:
# Scale the data
scaler = StandardScaler()
X_scaled=scaler.fit(X)

In [ ]:
# Train Model
model = IsolationForest(
    n_estimators=100,
    max_samples=200,
    contamination=0.01,
    max_features=1.0,
    bootstrap=False,
    n_jobs=-1, # Using available processors from computer
    random_state=42,
)
model.fit(X)

In [ ]:
y_pred = model.predict(X)

In [ ]:
pivoted_df.iloc[:,:5]

In [ ]:
X_df = pd.DataFrame(X)
y_pred_df = pd.DataFrame(y_pred)
final_df = pd.concat([pivoted_df.iloc[:,:5],X_df , y_pred_df], axis=1)
final_df.head()

In [ ]:
anomalies_detected=final_df[final_df[0] == -1]
#Creating a df with the list of anomalies
anomalies_df = pd.DataFrame(companies_with_anomalies, columns=['id', 'ticker', 'year', 'description'])

In [ ]:
match_anomalies = pd.merge(anomalies_detected, anomalies_df, on='ticker', how='inner')
match_anomalies[['ticker', 'year', 'description']]